In [1]:
import geopandas as gpd
import pandas as pd
import pandas as pd
import geopandas
import gmaps
import shapely.geometry as geom
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon
import pygeohash as gh
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time


ModuleNotFoundError: No module named 'gmaps'

In [29]:
def is_geohash_in_bounding_box(current_geohash, bbox_coordinates):
    """Checks if the box of a geohash is inside the bounding box

    :param current_geohash: a geohash
    :param bbox_coordinates: bounding box coordinates
    :return: true if the the center of the geohash is in the bounding box
    """

    coordinates = geohash.decode(current_geohash)
    geohash_in_bounding_box = (bbox_coordinates[0] < coordinates[0] < bbox_coordinates[2]) and (
            bbox_coordinates[1] < coordinates[1] < bbox_coordinates[3])
    return geohash_in_bounding_box
def build_geohash_box(current_geohash):
    """Returns a GeoJSON Polygon for a given geohash

    :param current_geohash: a geohash
    :return: a list representation of th polygon
    """

    b = geohash.bbox(current_geohash)
    polygon = [(b['w'], b['s']), (b['w'], b['n']), (b['e'], b['n']), (b['e'], b['s'],), (b['w'], b['s'])]
    return polygon
def compute_geohash_tiles(bbox_coordinates,GEOHASH_PRECISION):
    """Computes all geohash tile in the given bounding box

    :param bbox_coordinates: the bounding box coordinates of the geohashes
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding box, assuming the earth is flat ;)
    center_latitude = (bbox_coordinates[0] + bbox_coordinates[2]) / 2
    center_longitude = (bbox_coordinates[1] + bbox_coordinates[3]) / 2

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            if neighbor not in checked_geohashes and is_geohash_in_bounding_box(neighbor, bbox_coordinates):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes
def create_polygon(coordinates):
    polygon_geom = Polygon(coordinates)
    crs = {'init': 'epsg:4326'}
    polygon = gp.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])     
    return polygon.geometry # return polygon['geometry']
# this function is for save the geohashes into a json file 
def write_geohash_layer(geohashes,filename):
    """Writes a grid layer based on the geohashes

    :param geohashes: a list of geohashes
    """

    layer = MultiLineString([build_geohash_box(gh) for gh in geohashes])
    with open('result/' + filename + '.json', 'wb') as f:
        f.write(geojson.dumps(layer,sort_keys=True).encode('utf-8'))

def compute_geohash_tiles_from_polygon(polygon,GEOHASH_PRECISION):
    """Computes all hex tile in the given polygon

    :param polygon: the polygon
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding, assuming the earth is flat ;)
    
    #center_latitude = polygon.centroid.coords[0][0]
    #center_longitude = polygon.centroid.coords[0][1]
    
    center_latitude = polygon.centroid.coords[0][1]
    center_longitude = polygon.centroid.coords[0][0]

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            point = geometry.Point(geohash.decode(neighbor)[::-1])
            if neighbor not in checked_geohashes and polygon.contains(point):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes

def getNewDistance(lat1,lon1,degree,radius):
    R = 6378.1 #Radius of the Earth
    brng = math.radians(degree) #Bearing is 90 degrees converted to radians.
    d = radius #Distance in km

    #lat2  52.20444 - the lat result I'm hoping for
    #lon2  0.36056 - the long result I'm hoping for.

    lat1 = math.radians(lat1) #Current lat point converted to radians
    lon1 = math.radians(lon1) #Current long point converted to radians

    lat2 = math.asin( math.sin(lat1)*math.cos(d/R) + math.cos(lat1)*math.sin(d/R)*math.cos(brng))

    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1), math.cos(d/R)-math.sin(lat1)*math.sin(lat2))

    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)

    print(lat2)
    print(lon2)
    return lat2,lon2


def getDistance(lat1, long1, distance, bearing):
    
    try:
        # Define starting point.
        start = geopy.Point(lat1, long1)

        # Define a general distance object, initialized with a distance of 1 km.
        d = geopy.distance.VincentyDistance(kilometers = distance)

        # Use the `destination` method with a bearing of 0 degrees (which is north)
        # in order to go from point `start` 1 km to north.
        #print d.destination(point=start, bearing=0)
        return d.destination(point=start,bearing=bearing)
    except:
        pass

def quadraticpolygon(c,a,b):
    try:
        a = list(a)
        b = list(b)
        c = list(c)
        coords = np.array([a,b,c])
        f = scipy.interpolate.interp1d(coords[:, 0], coords[:, 1],kind='quadratic') #, kind='quadratic'
        fine_x = np.linspace(np.min(coords[:, 0]), np.max(coords[:, 0]),20)
        fine_y = f(fine_x)
        fine_coords = zip(fine_x, fine_y)
        #fine_line = LineString(fine_coords)
        myshape = geometry.Polygon(fine_coords)
        return myshape
    except:
        pass

    
def generatePolygon(lat1, long1, radius, angle1, angle2):
    import geopandas as gpd
    from shapely.geometry import Polygon
    lat_point_list = []
    lon_point_list = []
    point1 = getDistance(lat1, long1, radius, angle1)
    point2 = getDistance(lat1, long1, radius, angle2)
    
    point1_lat = point1[0]
    point1_lon = point1[1]
    
    point2_lat = point2[0]
    point2_lon = point2[1]
    
    lat_point_list.append(lat1)
    lat_point_list.append(point1_lat)
    lat_point_list.append(point2_lat)
    
    lon_point_list.append(long1)
    lon_point_list.append(point1_lon)
    lon_point_list.append(point2_lon)
    
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
    #print(polygon.geometry)
    
    return polygon.geometry
    
    
    

In [11]:
gdf = gpd.read_file('geofabrik/gis_osm_roads_free_1.shp')
gdf.head()


,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,4860427,5112,trunk,None,AA002,F,40,0,F,F,"LINESTRING (80.5488946 5.9451973, 80.548912 5...."
1,4860428,5122,residential,Alapalaawa Road,None,B,30,0,F,F,"LINESTRING (80.2112107 6.0482012, 80.2122339 6..."
2,4860436,5122,residential,Wijayananda Mawatha,None,B,30,0,F,F,"LINESTRING (80.2201955 6.0412283, 80.2196352 6..."
3,4860437,5122,residential,Wijayananda Mawatha,None,B,30,0,F,F,"LINESTRING (80.2194001 6.040748, 80.2188988000..."
4,4860438,5114,secondary,Dangedara Mawatha,B128,B,72,0,F,F,"LINESTRING (80.223033 6.0452337, 80.222978 6.0..."


In [12]:
gdf.fclass.unique()

array(['trunk', 'residential', 'secondary', 'tertiary', 'track',
       'primary', 'service', 'footway', 'living_street', 'track_grade2',
       'unclassified', 'trunk_link', 'primary_link', 'track_grade3',
       'steps', 'motorway', 'path', 'secondary_link', 'tertiary_link',
       'track_grade4', 'track_grade1', 'motorway_link', 'pedestrian',
       'track_grade5', 'bridleway', 'cycleway', 'unknown'], dtype=object)

In [34]:
cell_df = pd.read_csv("datathon.cell.details.csv", encoding="ISO-8859-1")
cell_df = cell_df[cell_df.AZIMUTH != 'No Planet ']
cell_df['AZIMUTH'] = cell_df['AZIMUTH'].astype(float)
cell_df['angle1'] = cell_df['AZIMUTH'] + 30
cell_df['angle2'] = cell_df['AZIMUTH'] - 30

cell_df = cell_df[cell_df.CELL_RADIUS != '¿3000']
cell_df = cell_df[cell_df.CELL_RADIUS != '2000m']
cell_df['CELL_RADIUS'] = cell_df['CELL_RADIUS'].astype(float)
cell_df['CELL_RADIUS'] = cell_df['CELL_RADIUS']/1000

cell_df = cell_df[cell_df.LATITUDE != 'No Planet Data']
cell_df['LATITUDE'] = cell_df['LATITUDE'].astype(float)
cell_df['LONGITUDE'] = cell_df['LONGITUDE'].astype(float)
cell_df = cell_df.dropna()



C:\Users\AC408\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
cell_df.head()

,SITE_ID,LOCATION_ID,CELL_NAME,TECHNOLOGY,TYPE,CELL_ID,LATITUDE,LONGITUDE,AZIMUTH,ANTENNA_HEIGHT,ALTITUDE,BEAMWIDTH,CELL_RADIUS,angle1,angle2
0,DKA0001,KA0001,Akurana-KA0001-D_1,2G,Outdoor,11557,7.362100,80.620654,0.0,50.0,532.83,65.75776,3.9,30.0,-30.0
1,DGA0001,GA0001,Ahungalla-GA0001-D_1,2G,Outdoor,18405,6.308892,80.036528,90.0,70.0,14.0,80.86603,4.7,120.0,60.0
2,DKA0147,KA0147,Akurana_North-KA0147-D_1,2G,Outdoor,11595,7.376493,80.616221,20.0,50.0,568.99,62.93914,3.5,50.0,-10.0
3,DGA0019,GA0019,Akmeemana-GA0019-D_1,2G,Outdoor,49868,6.078122,80.291020,20.0,60.0,103.03,86.26203,4.0,50.0,-10.0
4,DMR0095,MR0095,Akuressa_DTP-MR0095-D_1,2G,Outdoor,45644,6.103983,80.474231,20.0,60.0,71.23,67.38526,1.5,50.0,-10.0


In [36]:
cell_df.head()
groups = cell_df.groupby('CELL_ID')
print(cell_df['CELL_RADIUS'].nunique())
#print(cell_df['CELL_RADIUS'].unique())
#print(cell_df['CELL_ID'].unique().count())
#len(cell_df)

192


In [37]:
cell_df['polygon'] = cell_df.apply(lambda x: generatePolygon(x.LATITUDE,
                                                               x.LONGITUDE,
                                                               x.CELL_RADIUS,
                                                               x.angle1,
                                                               x.angle2), axis=1)



C:\Users\AC408\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:129: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


In [44]:
cell_df['geometry'] = cell_df['polygon']
cell_df.head()

,SITE_ID,LOCATION_ID,CELL_NAME,TECHNOLOGY,TYPE,CELL_ID,LATITUDE,LONGITUDE,AZIMUTH,ANTENNA_HEIGHT,ALTITUDE,BEAMWIDTH,CELL_RADIUS,angle1,angle2,polygon,geometry
0,DKA0001,KA0001,Akurana-KA0001-D_1,2G,Outdoor,11557,7.362100,80.620654,0.0,50.0,532.83,65.75776,3.9,30.0,-30.0,"POLYGON ((80.620654 7.3621, 80.63831699662623 ...","POLYGON ((80.620654 7.3621, 80.63831699662623 ..."
1,DGA0001,GA0001,Ahungalla-GA0001-D_1,2G,Outdoor,18405,6.308892,80.036528,90.0,70.0,14.0,80.86603,4.7,120.0,60.0,"POLYGON ((80.036528 6.308892, 80.0733121034101...","POLYGON ((80.036528 6.308892, 80.0733121034101..."
2,DKA0147,KA0147,Akurana_North-KA0147-D_1,2G,Outdoor,11595,7.376493,80.616221,20.0,50.0,568.99,62.93914,3.5,50.0,-10.0,"POLYGON ((80.6162207915649 7.37649308211153, 8...","POLYGON ((80.6162207915649 7.37649308211153, 8..."
3,DGA0019,GA0019,Akmeemana-GA0019-D_1,2G,Outdoor,49868,6.078122,80.291020,20.0,60.0,103.03,86.26203,4.0,50.0,-10.0,"POLYGON ((80.29102 6.078122, 80.3187017416307 ...","POLYGON ((80.29102 6.078122, 80.3187017416307 ..."
4,DMR0095,MR0095,Akuressa_DTP-MR0095-D_1,2G,Outdoor,45644,6.103983,80.474231,20.0,60.0,71.23,67.38526,1.5,50.0,-10.0,"POLYGON ((80.474231 6.103983, 80.4846118715339...","POLYGON ((80.474231 6.103983, 80.4846118715339..."


In [45]:
crs = {'init': 'epsg:4326'}
gdf_cell = GeoDataFrame(cell_df, crs=crs, geometry='geometry')

In [46]:
gdf_cell.head()

,SITE_ID,LOCATION_ID,CELL_NAME,TECHNOLOGY,TYPE,CELL_ID,LATITUDE,LONGITUDE,AZIMUTH,ANTENNA_HEIGHT,ALTITUDE,BEAMWIDTH,CELL_RADIUS,angle1,angle2,polygon,geometry
0,DKA0001,KA0001,Akurana-KA0001-D_1,2G,Outdoor,11557,7.362100,80.620654,0.0,50.0,532.83,65.75776,3.9,30.0,-30.0,"POLYGON ((80.620654 7.3621, 80.63831699662623 ...","POLYGON ((80.620654 7.3621, 80.63831699662623 ..."
1,DGA0001,GA0001,Ahungalla-GA0001-D_1,2G,Outdoor,18405,6.308892,80.036528,90.0,70.0,14.0,80.86603,4.7,120.0,60.0,"POLYGON ((80.036528 6.308892, 80.0733121034101...","POLYGON ((80.036528 6.308892, 80.0733121034101..."
2,DKA0147,KA0147,Akurana_North-KA0147-D_1,2G,Outdoor,11595,7.376493,80.616221,20.0,50.0,568.99,62.93914,3.5,50.0,-10.0,"POLYGON ((80.6162207915649 7.37649308211153, 8...","POLYGON ((80.6162207915649 7.37649308211153, 8..."
3,DGA0019,GA0019,Akmeemana-GA0019-D_1,2G,Outdoor,49868,6.078122,80.291020,20.0,60.0,103.03,86.26203,4.0,50.0,-10.0,"POLYGON ((80.29102 6.078122, 80.3187017416307 ...","POLYGON ((80.29102 6.078122, 80.3187017416307 ..."
4,DMR0095,MR0095,Akuressa_DTP-MR0095-D_1,2G,Outdoor,45644,6.103983,80.474231,20.0,60.0,71.23,67.38526,1.5,50.0,-10.0,"POLYGON ((80.474231 6.103983, 80.4846118715339...","POLYGON ((80.474231 6.103983, 80.4846118715339..."


In [48]:
intersections = gp.sjoin(gdf, gdf_cell, how='inner', op='intersects')

In [49]:
intersections.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,LATITUDE,LONGITUDE,AZIMUTH,ANTENNA_HEIGHT,ALTITUDE,BEAMWIDTH,CELL_RADIUS,angle1,angle2,polygon
0,4860427,5112,trunk,None,AA002,F,40,0,F,F,...,5.941108,80.502183,55.0,45.0,39.25,66.86275,10.0,85.0,25.0,"POLYGON ((80.502183 5.941108, 80.5921540067356..."
374,13881255,5115,tertiary,Welegoda Road,None,B,40,0,F,F,...,5.941108,80.502183,55.0,45.0,39.25,66.86275,10.0,85.0,25.0,"POLYGON ((80.502183 5.941108, 80.5921540067356..."
375,13881258,5115,tertiary,Kotikagoda Rajamaha Vihara Mawatha,None,B,60,0,F,F,...,5.941108,80.502183,55.0,45.0,39.25,66.86275,10.0,85.0,25.0,"POLYGON ((80.502183 5.941108, 80.5921540067356..."
376,13881265,5122,residential,Paramulla Road,None,B,30,0,F,F,...,5.941108,80.502183,55.0,45.0,39.25,66.86275,10.0,85.0,25.0,"POLYGON ((80.502183 5.941108, 80.5921540067356..."
539,23153251,5112,trunk,ColomboâMatara Road,AA002,B,60,0,F,F,...,5.941108,80.502183,55.0,45.0,39.25,66.86275,10.0,85.0,25.0,"POLYGON ((80.502183 5.941108, 80.5921540067356..."
